In [1]:
import os
from os.path import basename, dirname
import sys
import string
import glob

import numpy as np

from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu
from nltk import word_tokenize
import nltk
nltk.download('punkt')

from scipy import spatial
import gensim
from gensim.models import Word2Vec

#word2vec_path = "/share03/song/word2vec/en/model.txt"
#model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, unicode_errors='ignore')

lang1 = 'zh-CN'
lang2 = 'en'

[nltk_data] Downloading package punkt to /home/song/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
lang1_word2vec_path = "/mnt/elm/song/lrec/word2vec/{}/model.txt".format(lang1)
lang2_word2vec_path = "/mnt/elm/song/lrec/word2vec/{}/model.txt".format(lang2)
#lang1_model = gensim.models.KeyedVectors.load_word2vec_format(lang1_word2vec_path, unicode_errors='ignore')
#lang2_model = gensim.models.KeyedVectors.load_word2vec_format(lang2_word2vec_path, unicode_errors='ignore')
print ("word2vec model loaded")

word2vec model loaded


In [8]:
def get_vec_list(sentence, model):
    vecs = []
    for word in sentence:
        try:
            word = word.lower()
            vecs.append(model[word])
        except:
            #print ("no word in model: ", word)
            continue
    return vecs

def get_vecs_from_lines(sentences, model):
    vecs = []
    for sentence in sentences:
        vec_list = get_vec_list(sentence, model)
        vecs.append(vec_list)
    return vecs

def connect_vec_list(vec_list1, vec_lists2):
    return (vec_list1 + vec_lists2)

def get_average_vec(vec_list):
    avg_vec = np.zeros(100)
    #avg_vec = np.zeros(len(vec_list[0]))
    for vec in vec_list:
        avg_vec+=vec
    if (len(vec_list)!=0):
        avg_vec/=len(vec_list)
        return avg_vec
    else:
        print ("zero len")
        return np.zeros(100)
    
def get_cos_similarity(hy_vec, ref_vec):
    return (1 - spatial.distance.cosine(hy_vec, ref_vec))

def check_length(len1, len2):
    if (float(len1)/len2 < 0.5 or float(len2)/len1 < 0.5):
        return False
    else:
        return True

f = []
dp = []
flag = []
decide = [] 
match = []
lang1_len = 0
lang2_len = 0
lang1_vecs = []
lang2_vecs = []
trans_lang1_vecs =[]
trans_lang2_vecs =[]

def dp(i ,j):
    def get_sim(lang1_vec, trans_lang1_vec, lang2_vec, trans_lang2_vec, bidirection_flag):
        direction1_sim = get_cos_similarity(lang1_vec, trans_lang1_vec)
        direction2_sim = get_cos_similarity(lang2_vec, trans_lang2_vec)
        if (bidirection_flag==0):
            sim = direction2_sim
        if (bidirection_flag==1):
            sim = direction1_sim
        if (bidirection_flag==2):
            sim = (direction1_sim + direction2_sim)/2
        if (bidirection_flag==3):
            sim = max(direction1_sim , direction2_sim)
        return sim
        
    if (i>=lang1_len or j>=lang2_len):
        return 0 
    if (flag[i][j] == 1):
        return f[i][j]
    flag[i][j] = 1
    
    
    select_i_j2 = -1 
    select_i2_j = -1 
    select_i_j = -1 
    select_i_j0 = -1 
    select_i0_j = -1 
    # 1-2
    # location constrains
    if (location_flag == 1):
        if (abs((float(i)/lang1_len) - (float(j)/lang2_len))>0.1):
            return -1
    if (multimatch==True):
        if (j+1<lang2_len):
            len1 = len(trans_lang2_vecs[i])
            len2 = len(lang2_vecs[j])+len(lang2_vecs[j+1])
            if ((length_filter_flag == 0) or (check_length(len1, len2))):
                lang1_vec = get_average_vec(lang1_vecs[i])
                trans_lang1_vec = get_average_vec(connect_vec_list(trans_lang1_vecs[j], trans_lang1_vecs[j+1]))
                lang2_vec = get_average_vec(connect_vec_list(lang2_vecs[j], lang2_vecs[j+1]))
                trans_lang2_vec =  get_average_vec(trans_lang2_vecs[i])
                sim = get_sim(lang1_vec, trans_lang1_vec, lang2_vec, trans_lang2_vec, bidirection_flag)
                if (not ((sim_filter_flag == True) and (sim<sim_threshold))):
                    select_i_j2 = dp(i+1, j+2) + sim

        # 2-1
        if (i+1<lang1_len):
            len1 = len(trans_lang2_vecs[i])+len(trans_lang2_vecs[i+1])
            len2 = len(lang2_vecs[j])
            if ((length_filter_flag == 0) or (check_length(len1, len2))):
                lang1_vec = get_average_vec(connect_vec_list(lang1_vecs[i], lang1_vecs[i+1]))
                trans_lang1_vec = get_average_vec(trans_lang1_vecs[j])
                lang2_vec = get_average_vec(lang2_vecs[j])
                trans_lang2_vec = get_average_vec(connect_vec_list(trans_lang2_vecs[i], trans_lang2_vecs[i+1]))
                sim = get_sim(lang1_vec, trans_lang1_vec, lang2_vec, trans_lang2_vec, bidirection_flag)
                if (not ((sim_filter_flag == True) and (sim<sim_threshold))):
                    select_i2_j = dp(i+2, j+1) + sim 
    # 1-1
    lang1_vec = get_average_vec(lang1_vecs[i])
    trans_lang1_vec = get_average_vec(trans_lang1_vecs[j])
    lang2_vec = get_average_vec(lang2_vecs[j])
    trans_lang2_vec =  get_average_vec(trans_lang2_vecs[i])
    sim = get_sim(lang1_vec, trans_lang1_vec, lang2_vec, trans_lang2_vec, bidirection_flag)
    if (not ((sim_filter_flag == True) and (sim<sim_threshold))):
        select_i_j = dp(i+1, j+1) + sim
    
    # zero match en sentence
    select_i_j0 = dp(i, j+1)
    
    # zero match ja sentence
    select_i0_j = dp(i+1, j)
    
    best_score = -1
    best_index = -1
    #print (i, j)
    #print (select_i_j2, select_i2_j, select_i_j, select_i_j0, select_i0_j)
    for idx, score in enumerate([select_i_j2, select_i2_j, select_i_j, select_i_j0, select_i0_j], 1):
        if (score>best_score):
            best_score = score
            best_index = idx
    f[i][j] = best_score
    decide[i][j] = best_index 
    return f[i][j]
    
def align(lang1_lines, lang2_lines, trans_lang1_lines, trans_lang2_lines):
    # initialize dp array
    global f, flag, decide, match, lang1_len, lang2_len 
    global lang1_vecs, lang2_vecs, trans_lang1_vecs, trans_lang2_vecs
    lang1_len = len(lang1_lines)
    lang2_len = len(lang2_lines)
    f = [[0 for i in range(lang2_len)] for j in range(lang1_len)]
    flag = [[0 for i in range(lang2_len)] for j in range(lang1_len)]
    decide = [[0 for i in range(lang2_len)] for j in range(lang1_len)]
    match = [-1 for j in range(lang1_len)]
    
    # for English
    lang2_tokenized_lines = []
    for line in lang2_lines:
        new_line = word_tokenize(line.strip())
        lang2_tokenized_lines.append(new_line)

    trans_lang2_tokenized_lines = []
    for line in trans_lang2_lines:
        new_line = word_tokenize(line.strip())
        trans_lang2_tokenized_lines.append(new_line)

    lang2_len = len(lang2_tokenized_lines)
    trans_lang2_len = len(trans_lang2_tokenized_lines)
    lang1_vecs = get_vecs_from_lines(lang1_lines, lang1_model)
    trans_lang1_vecs = get_vecs_from_lines(trans_lang1_lines, lang1_model)
    lang2_vecs = get_vecs_from_lines(lang2_tokenized_lines, lang2_model)
    trans_lang2_vecs = get_vecs_from_lines(trans_lang2_tokenized_lines, lang2_model)
    

    dp(0, 0) # use avg_cos_mat
    
    
def get_align_from_decide():
    predict = []
    i = 0
    j = 0
    global lang1_len, lang2_len
    while (i<lang1_len and j<lang2_len):
        if (decide[i][j] == 1):
            predict.append(([i],[j,j+1]))
            i+=1
            j+=2
        elif (decide[i][j] == 2):
            predict.append(([i,i+1],[j]))
            i+=2
            j+=1
        elif (decide[i][j] == 3):
            predict.append(([i],[j]))
            i+=1
            j+=1
        elif (decide[i][j] == 4):
            j+=1
        elif (decide[i][j] == 5):
            i+=1
    return predict
    
def get_res_from_decide(lang1_lines, lang2_lines, trans_lang1_lines, trans_lang2_lines):
    sentence_pairs = []
    similarities = []
    i = 0
    j = 0
    global lang1_len, lang2_len
    while (i<lang1_len and j<lang2_len):
        lang1_sentence = ''
        lang2_sentence = ''
        trans_lang2_sentence = ''
        similarity = -1 
        if (decide[i][j] == 1):
            lang1_sentence = lang1_lines[i].strip()
            lang2_sentence = lang2_lines[j].strip() + ' '+ lang2_lines[j+1].strip()
            trans_lang2_sentence =trans_lang2_lines[i].strip()
            i+=1
            j+=2
        elif (decide[i][j] == 2):
            lang1_sentence = lang1_lines[i].strip() + ' '+ lang1_lines[i+1].strip()
            lang2_sentence = lang2_lines[j].strip()
            trans_lang2_sentence =trans_lang2_lines[i].strip()+ ' '+ trans_lang2_lines[i+1].strip()
            i+=2
            j+=1
        elif (decide[i][j] == 3):
            lang1_sentence = lang1_lines[i].strip()
            lang2_sentence = lang2_lines[j].strip()
            trans_lang2_sentence = trans_lang2_lines[i].strip()
            i+=1
            j+=1
        elif (decide[i][j] == 4):
            j+=1
        elif (decide[i][j] == 5):
            i+=1
        if (lang1_sentence!=''):
            vec1 = get_average_vec(get_vec_list(trans_lang2_sentence, lang2_model))
            vec2 = get_average_vec(get_vec_list(lang2_sentence, lang2_model))
            similarity = get_cos_similarity(vec1, vec2)
            sentence_pairs.append([lang1_sentence, lang2_sentence])
            similarities.append(similarity)
    return sentence_pairs, similarities

def save_results(sentence_pairs, lang1_file, lang2_file):
    with open(lang1_file, "w") as f1, open(lang2_file, "w") as f2:
        for i, sentence_pair in enumerate(sentence_pairs):
            lang1_sentence, lang2_sentence = sentence_pair
            f1.write(lang1_sentence.strip()+'\n')
            f2.write(lang2_sentence.strip()+'\n')

In [12]:
similarity_metrics = 'cs'
multimatch = 1 # 0: 1-1, 1: maximum 1-2
bidirection_flag = 0 # 0 use english, 1 use chinese, 2 average, 3 max
sim_filter_flag = 1
length_filter_flag = 1
sim_threshold = 0 
#sim_threshold = 0.92
location_flag = 0

def get_target_ids(target_name):
    target = []
    lines = open(target_name).readlines()
    for line in lines:
        t1, t2 = line.strip().split()
        lang1_id = [int(i)-1 for i in t1.split(',')]
        lang2_id = [int(i)-1 for i in t2.split(',')]
        target.append((lang1_id,lang2_id))
    return target

def res_compare(predict, target):
    def have_common(list1, list2):
        return (len(list(set(list1).intersection(list2)))>0)
    tot_num = len(target)
    partial_match = 0
    exact_match = 0
    for p in predict:
        exact_flag = 0
        partical_flag = 0
        for t in target:
            if (p[0] == t[0] and p[1]==t[1]):
                exact_flag = 1
            if have_common(p[0],t[0]) and have_common(p[1], t[1]):
                partical_flag = 1
        partial_match += partical_flag
        exact_match += exact_flag
    e_rate = float(exact_match)/tot_num
    p_rate = float(partial_match)/tot_num
    print (tot_num, exact_match, partial_match)
    return e_rate, p_rate

def alignment_test():
    test_basename = "762_03_siamese-network"
    lang1_name = "{}.{}.txt".format(test_basename, lang1)
    lang2_name = "{}.{}.txt".format(test_basename, lang2)
    trans_lang1_name = "{}.trans_{}.txt".format(test_basename, lang1)
    trans_lang2_name = "{}.trans_{}.txt".format(test_basename, lang2)
    target_name = "905_target_ids.txt"
    target = get_target_ids(target_name)
    #print (target)

    with open(lang1_name, "r") as f:
        lang1_lines = f.readlines()
    with open(lang2_name, "r") as f:
        lang2_lines = f.readlines()
    with open(trans_lang2_name, "r") as f:
        trans_lang2_lines = f.readlines()
    with open(trans_lang1_name, "r") as f:
        trans_lang1_lines = f.readlines()
        
    align(lang1_lines, lang2_lines, trans_lang1_lines, trans_lang2_lines)
    predict = get_align_from_decide()
    print (predict)
    exact_match_rate, partial_match_rate = res_compare(predict, target)
    print (exact_match_rate, partial_match_rate)

alignment_test()

[([0], [0]), ([1], [1]), ([2], [3, 4]), ([3], [5]), ([4], [6]), ([5], [7]), ([6], [8]), ([7], [9]), ([8], [10]), ([9], [11]), ([10], [12]), ([11], [14]), ([12], [15]), ([13], [16]), ([14], [17]), ([15], [18]), ([16], [19]), ([17], [23, 24]), ([18], [29]), ([19], [30]), ([20], [31, 32]), ([21], [33])]
22 18 22
0.8181818181818182 1.0


In [ ]:
#ef get_vec(sentence, model):
#    res = np.zeros(100)
#    num_of_words = len(sentence)
#    for word in sentence:
#        try:
#            res += model[word]
#        except:
#            num_of_words -= 1
#    res/=num_of_words
#    return res

#def get_vecs_from_lines(lines, model):
#    vecs = []
#    for line in lines:
#        vecs.append(get_vec(line, model))
#    return vecs


#def check_position_similarity(i, trans_len, j, en_len):
#    relative_trans_loc = float(i)/trans_len
#    relative_en_loc = float(j)/en_len
#    relative_err = abs(relative_trans_loc - relative_en_loc)
#    return relative_err